In [1]:
import pandas as pd
import numpy as np 
import xgboost as xgb
import patsy
import matplotlib.pyplot as plt
import sklearn.cross_validation as cv
from collections import Counter
import re
%matplotlib inline

In [2]:
def Gini(y_pred, dtrain):
    y_true = dtrain.get_label()
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]
    
    # sort rows on prediction column 
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:,0].argsort()][::-1,0]
    pred_order = arr[arr[:,1].argsort()][::-1,0]
    
    # get Lorenz curves
    L_true = np.cumsum(true_order) / np.sum(true_order)
    L_pred = np.cumsum(pred_order) / np.sum(pred_order)
    L_ones = np.linspace(0, 1, n_samples)
    
    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)
    
    # normalize to true Gini coefficient
    return 'Gini', -G_pred/G_true

## Train

In [3]:
import warnings
warnings.filterwarnings('ignore')
num_rounds = 10000
params = {  "objective": "reg:linear"
          , "eta": 0.003
          , "min_child_weight": 50
          , "subsample": 0.6
          , "colsample_bytree": 0.7
          , "scale_pos_weight": 1
          , "silent": 1
          , "max_depth": 9}

digs = re.compile(r'\d+')

cat_to_drop = []

tr = pd.read_csv('./train.csv', sep = ',', index_col = 'Id')
te =  pd.read_csv('./test.csv', sep = ',',index_col = 'Id')
labels = tr['Hazard'].values

full =  pd.concat(objs = [tr,te])

full_cat = full.select_dtypes(include = ['object'])
full_num = full.select_dtypes(exclude = ['object'])

cat_names = full_cat.columns    
form = ' + '.join(cat_names)
form += ' - 1'
x_dummies = patsy.dmatrix(form, full_cat, return_type='dataframe')
full_start = pd.concat([full_num, x_dummies], axis = 1)

for i in range(32):
    full_dummies = full_start.copy()
    full_dummies.drop(cat_to_drop, axis=1, inplace = True)
    
    split = np.isnan(full_dummies.Hazard)
    train = full_dummies[~split].drop('Hazard', axis=1).values
#   test  = full_dummies[split].drop('Hazard', axis=1).values
    feature_names = full_dummies.columns[1:]
    gini = []
    for j in range(3):
        folds = cv.StratifiedKFold(labels, n_folds = 5, shuffle=True)
        for train_ind, val_ind in folds: 
            xgtrain = xgb.DMatrix(train[train_ind,:], label=labels[train_ind])
            xgval = xgb.DMatrix(train[val_ind,:], label=labels[val_ind])
            watchlist = [(xgtrain, 'train'),(xgval, 'val')]
            model = xgb.train(params, xgtrain, num_rounds, watchlist,
                              feval= Gini, early_stopping_rounds=200,verbose_eval=False)
            gini.append(model.best_score)
    importance = Counter(model.get_fscore()).most_common()
    least_imp_id = re.findall(digs, importance[-1][0])
    least_imp_name = feature_names[least_imp_id]
    print('#vars: {} | CV repeats: {} | Mean Gini: {:.6f} | Std Gini: {:.6f} | Dropped: {} | Least Important: {}'.format(
            len(feature_names),len(gini), np.mean(gini),np.std(gini), cat_to_drop, least_imp_name))
    cat_to_drop.append(least_imp_name)

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2701]	train-Gini:-0.555148	val-Gini:-0.376333

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[1633]	train-Gini:-0.513446	val-Gini:-0.386152

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2438]	train-Gini:-0.547998	val-Gini:-0.387072

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2110]	train-Gini:-0.527062	val-Gini:-0.406181

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[3267]	train-Gini:-0.573539	val-Gini:-0.384079

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2324]	train-Gini:-0.542189	val-Gini:-0.388206

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2708]	train-Gini:-0.552793	val-Gini:-0.392432

Will train until val error hasn't decreased in 200 rounds.
Stopping. 

#vars: 96 | CV repeats: 15 | Mean Gini: -0.386362 | Std Gini: 0.008529 | Dropped: [] | Least Important: T1_V15[T.S]
#vars: 95 | CV repeats: 15 | Mean Gini: -0.386278 | Std Gini: 0.009037 | Dropped: ['T1_V15[T.S]'] | Least Important: T1_V15[T.F]

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2517]	train-Gini:-0.548143	val-Gini:-0.388303

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2175]	train-Gini:-0.532404	val-Gini:-0.406400

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2167]	train-Gini:-0.537534	val-Gini:-0.383361

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2330]	train-Gini:-0.541583	val-Gini:-0.371285

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2249]	train-Gini:-0.536241	val-Gini:-0.383613

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2192]	train-Gini:-0.535067	val-Gini:-0.377654

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2798]	train-Gini:-0.557260	val-Gini:-0.397858

Will train until val error hasn't decreased in 200 rounds.
Stopping. 


#vars: 94 | CV repeats: 15 | Mean Gini: -0.387025 | Std Gini: 0.009780 | Dropped: ['T1_V15[T.S]', 'T1_V15[T.F]'] | Least Important: T1_V9[T.C]

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2346]	train-Gini:-0.540712	val-Gini:-0.399358

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2280]	train-Gini:-0.539837	val-Gini:-0.379358

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2024]	train-Gini:-0.527321	val-Gini:-0.392754

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2557]	train-Gini:-0.550493	val-Gini:-0.382196

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[1387]	train-Gini:-0.504174	val-Gini:-0.379911

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2304]	train-Gini:-0.543105	val-Gini:-0.394642

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2465]	train-Gini:-0.545907	val-Gini:-0.383784

Will train until val error hasn't decreased in 200 rounds.
Stopping. 

...


#vars: 67 | CV repeats: 15 | Mean Gini: -0.385138 | Std Gini: 0.008351 | Dropped: ['T1_V15[T.S]', 'T1_V15[T.F]', 'T1_V9[T.C]', 'T1_V5[T.J]', 'T1_V16[T.F]', 'T1_V11[D]', 'T1_V11[K]', 'T1_V11[F]', 'T1_V16[T.Q]', 'T1_V11[N]', 'T1_V16[T.H]', 'T1_V16[T.O]', 'T1_V9[T.G]', 'T1_V16[T.G]', 'T1_V16[T.C]', 'T1_V7[T.D]', 'T1_V15[T.W]', 'T1_V16[T.P]', 'T1_V16[T.L]', 'T1_V5[T.D]', 'T1_V16[T.M]', 'T1_V16[T.D]', 'T1_V11[M]', 'T1_V11[E]', 'T2_V5[T.E]', 'T1_V12[T.D]', 'T1_V4[T.S]', 'T2_V5[T.D]', 'T2_V13[T.B]'] | Least Important: T1_V11[I]

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2122]	train-Gini:-0.536937	val-Gini:-0.410520

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2217]	train-Gini:-0.550425	val-Gini:-0.386898

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2594]	train-Gini:-0.563594	val-Gini:-0.381726

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2505]	train-Gini:-0.557268	val-Gini:-0.381653

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2189]	train-Gini:-0.546646	val-Gini:-0.372979

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2554]	train-Gini:-0.560256	val-Gini:-0.372322

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2051]	train-Gini:-0.541981	val-Gini:-0.396918

Will train until val error hasn't decreased in 200 rounds.
Stopping. 


#vars: 66 | CV repeats: 15 | Mean Gini: -0.386369 | Std Gini: 0.011069 | Dropped: ['T1_V15[T.S]', 'T1_V15[T.F]', 'T1_V9[T.C]', 'T1_V5[T.J]', 'T1_V16[T.F]', 'T1_V11[D]', 'T1_V11[K]', 'T1_V11[F]', 'T1_V16[T.Q]', 'T1_V11[N]', 'T1_V16[T.H]', 'T1_V16[T.O]', 'T1_V9[T.G]', 'T1_V16[T.G]', 'T1_V16[T.C]', 'T1_V7[T.D]', 'T1_V15[T.W]', 'T1_V16[T.P]', 'T1_V16[T.L]', 'T1_V5[T.D]', 'T1_V16[T.M]', 'T1_V16[T.D]', 'T1_V11[M]', 'T1_V11[E]', 'T2_V5[T.E]', 'T1_V12[T.D]', 'T1_V4[T.S]', 'T2_V5[T.D]', 'T2_V13[T.B]', 'T1_V11[I]'] | Least Important: T1_V15[T.D]

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[1895]	train-Gini:-0.529627	val-Gini:-0.391223

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[1930]	train-Gini:-0.534238	val-Gini:-0.383038

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2587]	train-Gini:-0.563186	val-Gini:-0.382058

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2492]	train-Gini:-0.560816	val-Gini:-0.385494

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2068]	train-Gini:-0.537409	val-Gini:-0.384800

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2116]	train-Gini:-0.538077	val-Gini:-0.385127

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2364]	train-Gini:-0.553681	val-Gini:-0.378134

Will train until val error hasn't decreased in 200 rounds.
Stopping. 


#vars: 65 | CV repeats: 15 | Mean Gini: -0.385189 | Std Gini: 0.007080 | Dropped: ['T1_V15[T.S]', 'T1_V15[T.F]', 'T1_V9[T.C]', 'T1_V5[T.J]', 'T1_V16[T.F]', 'T1_V11[D]', 'T1_V11[K]', 'T1_V11[F]', 'T1_V16[T.Q]', 'T1_V11[N]', 'T1_V16[T.H]', 'T1_V16[T.O]', 'T1_V9[T.G]', 'T1_V16[T.G]', 'T1_V16[T.C]', 'T1_V7[T.D]', 'T1_V15[T.W]', 'T1_V16[T.P]', 'T1_V16[T.L]', 'T1_V5[T.D]', 'T1_V16[T.M]', 'T1_V16[T.D]', 'T1_V11[M]', 'T1_V11[E]', 'T2_V5[T.E]', 'T1_V12[T.D]', 'T1_V4[T.S]', 'T2_V5[T.D]', 'T2_V13[T.B]', 'T1_V11[I]', 'T1_V15[T.D]'] | Least Important: T1_V4[T.G]


In [3]:
import warnings
warnings.filterwarnings('ignore')
num_rounds = 10000
params = {  "objective": "reg:linear"
          , "eta": 0.003
          , "min_child_weight": 50
          , "subsample": 0.6
          , "colsample_bytree": 0.7
          , "scale_pos_weight": 1
          , "silent": 1
          , "max_depth": 9}

digs = re.compile(r'\d+')

cat_to_drop = []

tr = pd.read_csv('./train.csv', sep = ',', index_col = 'Id')
te =  pd.read_csv('./test.csv', sep = ',',index_col = 'Id')
labels = tr['Hazard'].values

full =  pd.concat(objs = [tr,te])

full_cat = full.select_dtypes(include = ['object'])
full_num = full.select_dtypes(exclude = ['object'])

cat_names = full_cat.columns    
form = ' + '.join(cat_names)
form += ' - 1'
x_dummies = patsy.dmatrix(form, full_cat, return_type='dataframe')
full_start = pd.concat([full_num, x_dummies], axis = 1)

for i in range(32):
    full_dummies = full_start.copy()
    full_dummies.drop(cat_to_drop, axis=1, inplace = True)
    
    split = np.isnan(full_dummies.Hazard)
    train = full_dummies[~split].drop('Hazard', axis=1).values
#   test  = full_dummies[split].drop('Hazard', axis=1).values
    feature_names = full_dummies.columns[1:]
    gini = []
    for j in range(3):
        folds = cv.StratifiedKFold(labels, n_folds = 5, shuffle=True)
        for train_ind, val_ind in folds: 
            xgtrain = xgb.DMatrix(train[train_ind,:], label=labels[train_ind])
            xgval = xgb.DMatrix(train[val_ind,:], label=labels[val_ind])
            watchlist = [(xgtrain, 'train'),(xgval, 'val')]
            model = xgb.train(params, xgtrain, num_rounds, watchlist,
                              feval= Gini, early_stopping_rounds=200,verbose_eval=False)
            gini.append(model.best_score)
    importance = Counter(model.get_fscore()).most_common()
    least_imp_id = re.findall(digs, importance[-1][0])
    least_imp_name = feature_names[least_imp_id]
    print('#vars: {} | CV repeats: {} | Mean Gini: {:.6f} | Std Gini: {:.6f} | Dropped: {} | Least Important: {}'.format(
            len(feature_names),len(gini), np.mean(gini),np.std(gini), cat_to_drop, least_imp_name))
    cat_to_drop.append(least_imp_name)

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2701]	train-Gini:-0.555148	val-Gini:-0.376333

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[1633]	train-Gini:-0.513446	val-Gini:-0.386152

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2438]	train-Gini:-0.547998	val-Gini:-0.387072

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2110]	train-Gini:-0.527062	val-Gini:-0.406181

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[3267]	train-Gini:-0.573539	val-Gini:-0.384079

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2324]	train-Gini:-0.542189	val-Gini:-0.388206

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2708]	train-Gini:-0.552793	val-Gini:-0.392432

Will train until val error hasn't decreased in 200 rounds.
Stopping. 

#vars: 96 | CV repeats: 15 | Mean Gini: -0.386362 | Std Gini: 0.008529 | Dropped: [] | Least Important: T1_V15[T.S]
#vars: 95 | CV repeats: 15 | Mean Gini: -0.386278 | Std Gini: 0.009037 | Dropped: ['T1_V15[T.S]'] | Least Important: T1_V15[T.F]

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2517]	train-Gini:-0.548143	val-Gini:-0.388303

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2175]	train-Gini:-0.532404	val-Gini:-0.406400

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2167]	train-Gini:-0.537534	val-Gini:-0.383361

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2330]	train-Gini:-0.541583	val-Gini:-0.371285

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2249]	train-Gini:-0.536241	val-Gini:-0.383613

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2192]	train-Gini:-0.535067	val-Gini:-0.377654

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2798]	train-Gini:-0.557260	val-Gini:-0.397858

Will train until val error hasn't decreased in 200 rounds.
Stopping. 


#vars: 94 | CV repeats: 15 | Mean Gini: -0.387025 | Std Gini: 0.009780 | Dropped: ['T1_V15[T.S]', 'T1_V15[T.F]'] | Least Important: T1_V9[T.C]

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2346]	train-Gini:-0.540712	val-Gini:-0.399358

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2280]	train-Gini:-0.539837	val-Gini:-0.379358

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2024]	train-Gini:-0.527321	val-Gini:-0.392754

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2557]	train-Gini:-0.550493	val-Gini:-0.382196

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[1387]	train-Gini:-0.504174	val-Gini:-0.379911

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2304]	train-Gini:-0.543105	val-Gini:-0.394642

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2465]	train-Gini:-0.545907	val-Gini:-0.383784

Will train until val error hasn't decreased in 200 rounds.
Stopping. 

...


#vars: 67 | CV repeats: 15 | Mean Gini: -0.385138 | Std Gini: 0.008351 | Dropped: ['T1_V15[T.S]', 'T1_V15[T.F]', 'T1_V9[T.C]', 'T1_V5[T.J]', 'T1_V16[T.F]', 'T1_V11[D]', 'T1_V11[K]', 'T1_V11[F]', 'T1_V16[T.Q]', 'T1_V11[N]', 'T1_V16[T.H]', 'T1_V16[T.O]', 'T1_V9[T.G]', 'T1_V16[T.G]', 'T1_V16[T.C]', 'T1_V7[T.D]', 'T1_V15[T.W]', 'T1_V16[T.P]', 'T1_V16[T.L]', 'T1_V5[T.D]', 'T1_V16[T.M]', 'T1_V16[T.D]', 'T1_V11[M]', 'T1_V11[E]', 'T2_V5[T.E]', 'T1_V12[T.D]', 'T1_V4[T.S]', 'T2_V5[T.D]', 'T2_V13[T.B]'] | Least Important: T1_V11[I]

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2122]	train-Gini:-0.536937	val-Gini:-0.410520

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2217]	train-Gini:-0.550425	val-Gini:-0.386898

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2594]	train-Gini:-0.563594	val-Gini:-0.381726

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2505]	train-Gini:-0.557268	val-Gini:-0.381653

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2189]	train-Gini:-0.546646	val-Gini:-0.372979

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2554]	train-Gini:-0.560256	val-Gini:-0.372322

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2051]	train-Gini:-0.541981	val-Gini:-0.396918

Will train until val error hasn't decreased in 200 rounds.
Stopping. 


#vars: 66 | CV repeats: 15 | Mean Gini: -0.386369 | Std Gini: 0.011069 | Dropped: ['T1_V15[T.S]', 'T1_V15[T.F]', 'T1_V9[T.C]', 'T1_V5[T.J]', 'T1_V16[T.F]', 'T1_V11[D]', 'T1_V11[K]', 'T1_V11[F]', 'T1_V16[T.Q]', 'T1_V11[N]', 'T1_V16[T.H]', 'T1_V16[T.O]', 'T1_V9[T.G]', 'T1_V16[T.G]', 'T1_V16[T.C]', 'T1_V7[T.D]', 'T1_V15[T.W]', 'T1_V16[T.P]', 'T1_V16[T.L]', 'T1_V5[T.D]', 'T1_V16[T.M]', 'T1_V16[T.D]', 'T1_V11[M]', 'T1_V11[E]', 'T2_V5[T.E]', 'T1_V12[T.D]', 'T1_V4[T.S]', 'T2_V5[T.D]', 'T2_V13[T.B]', 'T1_V11[I]'] | Least Important: T1_V15[T.D]

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[1895]	train-Gini:-0.529627	val-Gini:-0.391223

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[1930]	train-Gini:-0.534238	val-Gini:-0.383038

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2587]	train-Gini:-0.563186	val-Gini:-0.382058

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2492]	train-Gini:-0.560816	val-Gini:-0.385494

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2068]	train-Gini:-0.537409	val-Gini:-0.384800

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2116]	train-Gini:-0.538077	val-Gini:-0.385127

Will train until val error hasn't decreased in 200 rounds.
Stopping. Best iteration:
[2364]	train-Gini:-0.553681	val-Gini:-0.378134

Will train until val error hasn't decreased in 200 rounds.
Stopping. 


#vars: 65 | CV repeats: 15 | Mean Gini: -0.385189 | Std Gini: 0.007080 | Dropped: ['T1_V15[T.S]', 'T1_V15[T.F]', 'T1_V9[T.C]', 'T1_V5[T.J]', 'T1_V16[T.F]', 'T1_V11[D]', 'T1_V11[K]', 'T1_V11[F]', 'T1_V16[T.Q]', 'T1_V11[N]', 'T1_V16[T.H]', 'T1_V16[T.O]', 'T1_V9[T.G]', 'T1_V16[T.G]', 'T1_V16[T.C]', 'T1_V7[T.D]', 'T1_V15[T.W]', 'T1_V16[T.P]', 'T1_V16[T.L]', 'T1_V5[T.D]', 'T1_V16[T.M]', 'T1_V16[T.D]', 'T1_V11[M]', 'T1_V11[E]', 'T2_V5[T.E]', 'T1_V12[T.D]', 'T1_V4[T.S]', 'T2_V5[T.D]', 'T2_V13[T.B]', 'T1_V11[I]', 'T1_V15[T.D]'] | Least Important: T1_V4[T.G]
